In [19]:
import os
import pandas as pd

In [27]:
BASE_PATH = '/'.join(os.getcwd().split('/')[:-1]) + '/'
RESULTS_PATH = BASE_PATH +  'results/'
EMISSIONS_FILE = 'emissions.csv'
METRICS_FILE = 'metrics.csv'
PARAMS_FILE = 'params.csv'

In [29]:
experiments = []

for d in next(os.walk(RESULTS_PATH))[1]:
	for m in next(os.walk(RESULTS_PATH + d))[1]:
		experiments.append(os.path.join(d, m))

experiments = sorted(experiments)

In [32]:
for i, v in enumerate(experiments):
	print(f'{i+1:<4}{v:<15}')

1   movielens/CFKG 
2   movielens/Pop  


In [33]:
emissions = pd.read_csv(os.path.join(RESULTS_PATH, experiments[0], EMISSIONS_FILE))
metrics = pd.read_csv(os.path.join(RESULTS_PATH, experiments[0], METRICS_FILE))
params = pd.read_csv(os.path.join(RESULTS_PATH, experiments[0], PARAMS_FILE))

In [34]:
emissions.head()

,timestamp,project_name,run_id,duration,emissions,emissions_rate,cpu_power,gpu_power,ram_power,cpu_energy,...,cpu_count,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,pue
0,2023-11-23T19:39:49,MOVIELENS_CFKG_PARAMS_TUNING_1700764630,c120e1ce-d520-4b17-8b0e-ce11d674db41,151.310700,0.000118,7.815694e-07,0.4886,0.0283,0.496891,0.000334,...,12,Apple M2 Max,1,Apple M2 Max,16.3097,41.2325,64.0,process,N,1.0
1,2023-11-23T19:42:36,MOVIELENS_CFKG_PARAMS_TUNING_1700764630,36c5e92d-25c9-48b1-b430-0177f2aede90,149.740166,0.000115,7.670373e-07,0.0651,0.0251,0.551628,0.000324,...,12,Apple M2 Max,1,Apple M2 Max,16.3097,41.2325,64.0,process,N,1.0
2,2023-11-23T19:45:22,MOVIELENS_CFKG_PARAMS_TUNING_1700764630,ec9ef4cf-0a01-446e-88de-54fabfb80ac3,146.672310,0.000107,7.265401e-07,0.0734,0.0126,0.000000,0.000300,...,12,Apple M2 Max,1,Apple M2 Max,16.3097,41.2325,64.0,process,N,1.0
3,2023-11-23T19:48:11,MOVIELENS_CFKG_PARAMS_TUNING_1700764630,b103a611-1d3d-423e-a31a-49f95e3955f3,149.991545,0.000118,7.884430e-07,0.0373,0.0254,0.573275,0.000333,...,12,Apple M2 Max,1,Apple M2 Max,16.3097,41.2325,64.0,process,N,1.0


In [35]:
metrics.head()

,recall@10,mrr@10,ndcg@10,hit@10,map@10,precision@10,gauc,itemcoverage@10,averagepopularity@10,giniindex@10,shannonentropy@10,tailpercentage@10,run_id,project_name
0,0.0987,0.3387,0.1789,0.5962,0.1005,0.145,0.8983,0.1581,1761.0495,0.9819,0.0076,0.0,c120e1ce-d520-4b17-8b0e-ce11d674db41,MOVIELENS_CFKG_PARAMS_TUNING_1700764630
1,0.0945,0.2834,0.1556,0.5778,0.0825,0.131,0.8929,0.2426,1102.3454,0.9291,0.0066,0.0,36c5e92d-25c9-48b1-b430-0177f2aede90,MOVIELENS_CFKG_PARAMS_TUNING_1700764630
2,0.0987,0.3387,0.1789,0.5962,0.1005,0.145,0.8983,0.1581,1761.0495,0.9819,0.0076,0.0,ec9ef4cf-0a01-446e-88de-54fabfb80ac3,MOVIELENS_CFKG_PARAMS_TUNING_1700764630
3,0.1174,0.3395,0.1915,0.6444,0.1064,0.159,0.9114,0.3397,1063.5906,0.9049,0.0049,0.0,b103a611-1d3d-423e-a31a-49f95e3955f3,MOVIELENS_CFKG_PARAMS_TUNING_1700764630


In [36]:
params.head()

,gpu_id,worker,use_gpu,seed,state,reproducibility,data_path,checkpoint_dir,show_progress,save_dataset,...,entity_kg_num_interval,relation_kg_num_interval,benchmark_filename,MODEL_TYPE,encoding,dataset,model,device,run_id,project_name
0,0,0,True,42,INFO,True,data/,saved/movielens/CFKG,True,False,...,"[0,inf)","[0,inf)",NaN,ModelType.KNOWLEDGE,utf-8,movielens,CFKG,mps,c120e1ce-d520-4b17-8b0e-ce11d674db41,MOVIELENS_CFKG_PARAMS_TUNING_1700764630
1,0,0,True,42,INFO,True,data/,saved/movielens/CFKG,True,False,...,"[0,inf)","[0,inf)",NaN,ModelType.KNOWLEDGE,utf-8,movielens,CFKG,mps,36c5e92d-25c9-48b1-b430-0177f2aede90,MOVIELENS_CFKG_PARAMS_TUNING_1700764630
2,0,0,True,42,INFO,True,data/,saved/movielens/CFKG,True,False,...,"[0,inf)","[0,inf)",NaN,ModelType.KNOWLEDGE,utf-8,movielens,CFKG,mps,ec9ef4cf-0a01-446e-88de-54fabfb80ac3,MOVIELENS_CFKG_PARAMS_TUNING_1700764630
3,0,0,True,42,INFO,True,data/,saved/movielens/CFKG,True,False,...,"[0,inf)","[0,inf)",NaN,ModelType.KNOWLEDGE,utf-8,movielens,CFKG,mps,b103a611-1d3d-423e-a31a-49f95e3955f3,MOVIELENS_CFKG_PARAMS_TUNING_1700764630
